In [ ]:
print("Shree Ram")

Shree Ram


In [ ]:
!pip install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00


In [ ]:
import anvil.server

anvil.server.connect("server_GZMJ3FGREXONNV7RQ3PFQ6DI-BX4UIUOEXNLMKCT7")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
BERT Embeddings with Random Forest Classification

This script uses BERT embeddings with a Random Forest classifier
for sentiment analysis and sarcasm detection.
"""

import os
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE


def download_datasets():
    """Download required datasets if they don't exist."""
    datasets = [
        ("https://gist.githubusercontent.com/Phani-2608/1610e06ac549fc3a17c1027eeb9275c1/raw/f07273b926b4a8ff45394b36bfa2b9e969098a1a/consolidated.csv", "consolidated csv")
    ]

    for url, filename in datasets:
        if not os.path.exists(filename):
            print(f"Downloading {filename}...")
            os.system(f"wget {url}")


def load_data():
    df = pd.read_csv("consolidated.csv")
    return df


def get_bert_features(text, tokenizer, model):
    """Extract BERT features for a single text, handling CPU/GPU correctly."""
    # Tokenize input on CPU
    encoding = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )
    # Move inputs to the same device as the model
    device = next(model.parameters()).device
    encoding = {k: v.to(device) for k, v in encoding.items()}

    # Forward pass without gradient tracking
    with torch.no_grad():
        outputs = model(**encoding)
        # Mean-pool the token embeddings to get a sentence vector
        emb = outputs.last_hidden_state.mean(dim=1).squeeze()

    # Detach from graph, move to CPU, convert to NumPy
    return emb.detach().cpu().numpy()


def process_data(df, tokenizer, model):
    """Process data to extract BERT embeddings for all sentences."""
    print("Extracting BERT embeddings...")
    embeddings = []

    for i, text in enumerate(df['sentence']):
        if i % 100 == 0:
            print(f"Processing {i}/{len(df)} sentences...")
        emb = get_bert_features(text, tokenizer, model)
        embeddings.append(emb)

    return np.vstack(embeddings)


def apply_smote(X, y):
    """Apply SMOTE to handle class imbalance."""
    print("Applying SMOTE to balance classes...")
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    return smote.fit_resample(X, y)


def train_and_evaluate_model(X_train, X_test, y_train, y_test):
    """Train Random Forest model and evaluate its performance."""
    print("Training Random Forest classifier...")
    clf = RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        verbose=1
    )

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:\n", report)

    return clf, accuracy, report


def main():
    """Main orchestration: download, load, embed, balance, train, evaluate."""
    # Pick GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    download_datasets()

    print("Loading BERT model and tokenizer...")
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name).to(device)

    df = load_data()
    print(f"Dataset shape: {df.shape}")
    print(f"Class distribution:\n{df['label'].value_counts()}")

    X = process_data(df, tokenizer, model)
    y = df['label'].values

    X_resampled, y_resampled = apply_smote(X, y)
    print(f"Shape after SMOTE: X={X_resampled.shape}, y={y_resampled.shape}")

    X_train, X_test, y_train, y_test = train_test_split(
        X_resampled, y_resampled, test_size=0.2, random_state=42
    )

    model, _, _ = train_and_evaluate_model(X_train, X_test, y_train, y_test)

    print("Process completed successfully!")

    return model


if __name__ == "__main__":
    model = main()

Using device: cpu
Loading BERT model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Dataset shape: (55328, 2)
Class distribution:
label
0    29970
1    25358
Name: count, dtype: int64
Extracting BERT embeddings...
Processing 0/55328 sentences...
Processing 100/55328 sentences...
Processing 200/55328 sentences...
Processing 300/55328 sentences...
Processing 400/55328 sentences...
Processing 500/55328 sentences...
Processing 600/55328 sentences...
Processing 700/55328 sentences...
Processing 800/55328 sentences...
Processing 900/55328 sentences...
Processing 1000/55328 sentences...
Processing 1100/55328 sentences...
Processing 1200/55328 sentences...
Processing 1300/55328 sentences...
Processing 1400/55328 sentences...
Processing 1500/55328 sentences...
Processing 1600/55328 sentences...
Processing 1700/55328 sentences...
Processing 1800/55328 sentences...
Processing 1900/55328 sentences...
Processing 2000/55328 sentences...
Processing 2100/55328 sentences...
Processing 2200/55328 sentences...
Processing 2300/55328 sentences...
Processing 2400/55328 sentences...
Process

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s


Accuracy: 0.9578
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.97      0.96      5998
           1       0.97      0.94      0.96      5990

    accuracy                           0.96     11988
   macro avg       0.96      0.96      0.96     11988
weighted avg       0.96      0.96      0.96     11988

Process completed successfully!


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.7s finished


In [ ]:
def predict_sarcasm(query):

  query_emb = get_bert_features(query, BertTokenizer.from_pretrained('bert-base-uncased'), BertModel.from_pretrained('bert-base-uncased'))
  res = model.predict(query_emb.reshape(1,-1))[0]
  if res:
    return 'The query is sarcastic'
  else:
    return 'the query is not sarcastic'



In [ ]:
@anvil.server.callable
def detect_sar_1(input):
  query = 'He is riding a bike'
  sarcasm_text= input
  result = predict_sarcasm(sarcasm_text)

  return sarcasm_text +" : " + result +" from BERT"

In [ ]:
anvil.server.wait_forever()

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_job